<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Finetuning/blob/main/50-Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

In [1]:
import gradio as gr

# 인터페이스에서 이벤트가 발생되면 실행되는 함수
def greet(name):
    return "안녕하세요 " + name + "선생님!"

In [ ]:
# gr.Interface(fn=greet, inputs="text", outputs="text").launch(server_name='0.0.0.0')

In [2]:
# inputs로 TextBox 타입의 컴포넌트, outputs로 TextBox 타입의 컴포넌트가 하나씩 적용
demo = gr.Interface(fn=greet, inputs="text", outputs="text")

In [ ]:
# demo.launch(server_name='0.0.0.0')

In [ ]:


# 서버 어플리케이션은 모든 네트워크 인터페이스에 바인딩한다
demo.launch(server_name='0.0.0.0')

In [1]:
import gradio as gr

def greet(name):
    return "안녕하세요 " + name + "선생님!"

demo = gr.Interface(
    fn=greet,
    inputs=gr.Textbox(lines=1, placeholder="여기에 이름을 쓰세요."), # TextBox의 속성을 변경
    outputs="text",
)

demo.launch(server_name='0.0.0.0')

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a0a560a2d1164f6ca2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
import gradio as gr

In [2]:
# 여기서의 input이 인터페이스의 input(순서대로)
def greet(name, is_morning, temperature):
    salutation = "좋은 아침입니다" if is_morning else "좋은 저녁입니다"
    greeting = f"{salutation} {name}선생님. 오늘은 섭씨 {temperature}도입니다."
    fahrenheit = (temperature - 32) * 5 / 9
    return greeting, (temperature * 9 / 5) + 32

In [3]:
# 리턴은 인터페이스의 output(순서대로)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "checkbox", gr.Slider(0, 100)],
    outputs=["text", "number"],
)

In [ ]:
demo.launch(server_name='0.0.0.0')


## ChatGPT 작품

In [ ]:
import gradio as gr

def custom_func(name, is_morning, temperature):
    # 예시: 입력값들을 각각 출력
    greeting = f"Good morning, {name}!" if is_morning else f"Hello, {name}!"
    return greeting, temperature

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            name = gr.Textbox(label="name")
            is_morning = gr.Checkbox(label="is_morning")
            temperature = gr.Slider(0, 100, step=1, label="temperature")
            clear_btn = gr.Button("Clear")
            submit_btn = gr.Button("Submit", variant="primary")
        with gr.Column():
            output0 = gr.Textbox(label="output 0")
            output1 = gr.Textbox(label="output 1", value="0")
            flag_btn = gr.Button("Flag")

    # 버튼 동작
    submit_btn.click(
        custom_func,
        inputs=[name, is_morning, temperature],
        outputs=[output0, output1]
    )
    clear_btn.click(
        lambda: ("", "", 0),  # 입력값 초기화
        inputs=[],
        outputs=[name, is_morning, temperature]
    )

demo.launch()

## ChatGPT 2번째 작품

In [ ]:
import gradio as gr

def custom_func(name, is_morning, temperature):
    # 출력 0: 인삿말
    greeting = f"Good morning, {name}!" if is_morning else f"Hello, {name}!"
    # 출력 1: 섭씨 -> 화씨 변환
    fahrenheit = temperature * 9 / 5 + 32
    return greeting, str(fahrenheit)

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            name = gr.Textbox(label="name")
            is_morning = gr.Checkbox(label="is_morning")
            temperature = gr.Slider(0, 100, step=1, label="temperature")
            clear_btn = gr.Button("Clear")
            submit_btn = gr.Button("Submit", variant="primary")
        with gr.Column():
            output0 = gr.Textbox(label="output 0")
            output1 = gr.Textbox(label="output 1", value="0")
            flag_btn = gr.Button("Flag")

    # Submit 버튼: 결과 출력
    submit_btn.click(
        custom_func,
        inputs=[name, is_morning, temperature],
        outputs=[output0, output1]
    )
    # Clear 버튼: 입력값 초기화
    clear_btn.click(
        lambda: ("", False, 0),  # 이름, is_morning, 온도 초기화
        inputs=[],
        outputs=[name, is_morning, temperature]
    )

demo.launch()


In [1]:
import numpy as np
import gradio as gr

In [2]:
def sepia(input_img):
    edge_detect_filter = np.array([
    [-1, -1, -1],
    [-1,  8, -1],
    [-1, -1, -1]
])
    sepia_img = input_img.dot(edge_detect_filter)
    sepia_img /= sepia_img.max()
    return sepia_img

# gr.Image는 이미지만 전문으로 입력 또는 출력하는 컴포넌트
demo = gr.Interface(sepia, gr.Image(), "image")
demo.launch(server_name='0.0.0.0')

## ChatGPT 3번째 작품

In [1]:
import gradio as gr
import numpy as np
from PIL import Image
from scipy.ndimage import convolve

def edge_detect_onoff(image, threshold=128):
    if image is None:
        return None

    # 1. 그레이스케일 변환
    gray = image.convert("L")  # 0~255 단일 채널

    # 2. 바이너리 변환 (on/off)
    np_gray = np.array(gray)
    binary = np.where(np_gray > threshold, 255, 0).astype(np.uint8)

    # 3. 엣지 디텍트 필터
    edge_filter = np.array([
        [-1, -1, -1],
        [-1,  8, -1],
        [-1, -1, -1]
    ])
    edges = convolve(binary, edge_filter)
    edges = np.clip(edges, 0, 255).astype(np.uint8)

    # 4. 결과를 PIL 이미지로 변환
    return Image.fromarray(edges)

demo = gr.Interface(
    fn=edge_detect_onoff,
    inputs=gr.Image(type="pil", label="원본 이미지 업로드"),
    outputs=gr.Image(type="pil", label="Edge Detected (On/Off) Image"),
    title="Edge Detect (On/Off) Gradio Demo",
    description="입력 이미지를 흑백 on/off(바이너리)로 변환 후 edge detection 결과를 보여줍니다."
)

demo.launch(server_name='0.0.0.0')


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://436f474371d37fa2a5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
import gradio as gr
import numpy as np
from PIL import Image
from scipy.ndimage import convolve

def edge_detect(image):
    # 이미지가 None이면 바로 반환
    if image is None:
        return None

    # Gradio에서 입력된 이미지는 PIL.Image 또는 numpy.ndarray
    if isinstance(image, Image.Image):
        img_np = np.array(image.convert("RGB"))
    elif isinstance(image, np.ndarray):
        img_np = image
    else:
        raise ValueError("Unsupported image type")

    # 엣지 디텍트 필터 정의
    edge_filter = np.array([
        [-1, -1, -1],
        [-1,  8, -1],
        [-1, -1, -1]
    ])

    # 각 채널별로 엣지 필터 적용
    result = np.zeros_like(img_np)
    for c in range(3):
        result[..., c] = convolve(img_np[..., c], edge_filter)
    result = np.clip(result, 0, 255).astype(np.uint8)

    # numpy array -> PIL Image로 변환
    return Image.fromarray(result)

demo = gr.Interface(
    fn=edge_detect,
    inputs=gr.Image(type="pil", label="원본 이미지 업로드"),
    outputs=gr.Image(type="pil", label="Edge Detected Image"),
    title="Edge Detect Gradio Demo",
    description="이미지를 업로드하면 Edge Detection 결과를 보여줍니다."
)

demo.launch(server_name='0.0.0.0')

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2b9a05a546fef2a0e9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
import gradio as gr
import numpy as np
from PIL import Image
from scipy.ndimage import convolve
import os

def edge_detect(input_img_path):
    # 파일 확장자 구분
    _, ext = os.path.splitext(input_img_path)
    ext = ext.lower()

    # 이미지 읽기 (RGB로 통일)
    img = Image.open(input_img_path).convert("RGB")
    img_np = np.array(img)

    # 파일 타입에 따라 분기 처리
    if ext in ['.png']:
        # png 파일이면: 예시 - Edge Detect + 반전(invert) 추가
        processed_img = edge_detect_filter(img_np)
        processed_img = 255 - processed_img   # 색 반전(예시)
    elif ext in ['.jpg', '.jpeg']:
        # jpg/jpeg 파일이면: 예시 - Edge Detect만 적용
        processed_img = edge_detect_filter(img_np)
    else:
        # 기타 확장자: 그냥 Edge Detect만 적용
        processed_img = edge_detect_filter(img_np)

    return processed_img

def edge_detect_filter(input_img):
    edge_filter = np.array([
        [-1, -1, -1],
        [-1,  8, -1],
        [-1, -1, -1]
    ])
    if input_img.ndim == 3 and input_img.shape[2] == 3:
        result = np.zeros_like(input_img)
        for c in range(3):
            result[..., c] = convolve(input_img[..., c], edge_filter)
        result = np.clip(result, 0, 255).astype(np.uint8)
    else:
        result = convolve(input_img, edge_filter)
        result = np.clip(result, 0, 255).astype(np.uint8)
    return result

demo = gr.Interface(
    fn=edge_detect,
    inputs=gr.Image(type="filepath"),      # 파일 경로로 입력
    outputs=gr.Image(type="numpy"),        # 이미지로 출력
    title="Edge Detect: PNG/JPG Handling Demo",
    description="Upload a PNG or JPG image. PNG will be edge-detected and inverted, JPG will be edge-detected only."
)

demo.launch(server_name='0.0.0.0')

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7fbedd3441ab77c9a7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://68465b07efa95ff012.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import random
import gradio as gr

def random_response(message, history):
    return random.choice(["네", "아니오"])

demo = gr.ChatInterface(random_response)

# 히스토리 관리까지 해주는 신규 컴포넌트(gradio가 최신버전으로 업그레이드 되어야 함)
# Interface 또는 ChatInterface 둘중 택일(Blocks를 쓰지 않는 경우, 정형화된 입력)
demo.launch(server_name='0.0.0.0')

In [ ]:
import gradio as gr

def greet(name):
    return "안녕하세요 " + name + "씨!"

with gr.Blocks() as demo:
    name = gr.Textbox(label="Name")
    output = gr.Textbox(label="Output Box")
    greet_btn = gr.Button("Greet")
    greet_btn.click(fn=greet, inputs=name, outputs=output, api_name="greet")

demo.launch(server_name='0.0.0.0')

In [ ]:
import numpy as np
import gradio as gr


def flip_text(x):
    return x[::-1]


def flip_image(x):
    return np.fliplr(x)


with gr.Blocks() as demo:
    gr.Markdown("Flip text or image files using this demo.")
    with gr.Tab("Flip Text"):
        text_input = gr.Textbox()
        text_output = gr.Textbox()
        text_button = gr.Button("Flip")
    with gr.Tab("Flip Image"):
        with gr.Row():
            image_input = gr.Image()
            image_output = gr.Image()
        image_button = gr.Button("Flip")

    with gr.Accordion("Open for More!"):
        gr.Markdown("Look at me...")

    text_button.click(flip_text, inputs=text_input, outputs=text_output)
    image_button.click(flip_image, inputs=image_input, outputs=image_output)

demo.launch(server_name='0.0.0.0')


In [ ]:
import gradio as gr

def calculator(num1, operation, num2):
    if operation == "add":
        return num1 + num2
    elif operation == "subtract":
        return num1 - num2
    elif operation == "multiply":
        return num1 * num2
    elif operation == "divide":
        if num2 == 0:
            raise gr.Error("Cannot divide by zero!")
        return num1 / num2

# 함수, inputs, outputs, 그외 example도 간단하게 추가 가능

demo = gr.Interface(
    calculator,
    [
        "number",
        gr.Radio(["add", "subtract", "multiply", "divide"]),
        "number"
    ],
    "number",
    examples=[
        [5, "add", 3],
        [4, "divide", 2],
        [-4, "multiply", 2.5],
        [0, "subtract", 1.2],
    ],
    title="Toy Calculator",
    description="Here's a sample toy calculator. Allows you to calculate things like $2+2=4$",
)

demo.launch(server_name='0.0.0.0')
